<a href="https://colab.research.google.com/github/Dasher0n/Data-cleaning/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Librerias y conexión

In [1]:
from google.colab import drive
import os
import pandas as pd
import re

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


# Extracción de datos

### Funciones

In [3]:
# Función para reordenar las columnas de un dataframe
def reordenar_columnas(df):
    # Identificar la columna de número de cuenta
    columna_numero_cuenta = None
    for col in df.columns:
        if "de cuenta" in col.lower():
            columna_numero_cuenta = col
            break

    # Asegurarse de que la columna de número de cuenta fue encontrada
    if columna_numero_cuenta:
        # Reordenar las columnas
        columnas_ordenadas = ['Marca temporal', columna_numero_cuenta] + [col for col in df.columns if col not in ['Marca temporal', columna_numero_cuenta]]
        df = df[columnas_ordenadas]

    return df

def convertir_a_string(df):
    for col in df.columns[1:]:  # Omitimos la primera columna
        df[col] = df[col].astype(str)  # Convertimos cada columna a str de manera individual
    return df


### Importaciónd de datos desde drive

In [4]:
folder_path = "/content/drive/My Drive/Cuestionarios"
# Listar los archivos de Excel en la carpeta
files = os.listdir(folder_path)
excel_files = [f for f in files if f.endswith('.xlsx') or f.endswith('.xls')]

# Diccionarios para almacenar los DataFrames y la relación número-nombre
dataframes = {}
file_map = {}

for i, file in enumerate(excel_files, start=1):
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path, header=0)  # Usa la primera fila como encabezado
    dataframes[i] = df  # Guarda el DataFrame con un número
    file_map[i] = file  # Guarda la relación número-nombre

# Mostrar el mapeo entre números y nombres originales
print(" Mapeo de archivos:")
for key, value in file_map.items():
    print(f"{key}: {value}")

 Mapeo de archivos:
1: CaminoProfesionalMAC1.xlsx
2: Finanzas Transparentes (Respuestas).xlsx
3: Habilidades de MAC (respuestas).xlsx
4: Impacto del Horario en el Desempeño y Bienestar de los Estudiantes de Minería de Datos (respuestas).xlsx
5: Perfil Académico (respuestas).xlsx
6: Rendimiento académico (respuestas).xlsx
7: Uso de Tecnología y Redes Sociales.xlsx


### Muestra de dataframe

In [5]:
# Acceder a un DataFrame específico por número
dataframes[3].iloc[:,[3,4,5,6,7,8,9]]

,2. ¿Cómo describirías tu capacidad para explicar ideas complejas a personas que no son expertas en tu área?,3. ¿Qué tan cómodo te sientes trabajando en equipo?,4. ¿Cuál ha sido tu mayor desafío al trabajar en equipo y cómo lo superaste?,5.¿Sientes que la carrera de MAC te ha preparado adecuadamente para el trabajo en equipo?,6. ¿Qué tan fácil o difícil te resulta establecer nuevas conexiones?,7. ¿Cómo te sientes al tener que hablar frente a una gran audiencia?,8. ¿Qué técnicas usas para manejar el estrés y mantener la calma en situaciones difíciles?
0,mala,regular,la comunicacion,no,es dificil,no me gusta hablar enfrente de una gran audiencia,no tengo tecnicas
1,No muy buena,Regular,Comunicarme efectivamente con los demás y he t...,No,Me resulta levemente difícil,Nervioso,Técnicas de respiración.
2,Sigo trabajando en el manejo del lenguaje al e...,Cómodo,"Diferencia de ideas, negociando",No tanto,Depende del tipo de conexión,Nervioso,Ejercicios de respiración
3,Buena,Muy cómoda,"Lidiar con gente que no conoce el tema, explic...",No,Muy fácil,Nerviosa,Terapia
4,He desarrollado una habilidad hacia ello con a...,muy cómoda,"trabajar con alguien que no me agrada, me fuer...","no, el intento con actividades blandas se sien...",facíl,nerviosa,Repetirme muchas veces que puedo hacerlo
5,Puede mejorar,Bastante,Trabajar con personas complicadas,"En parte, puede mejorear",Algo difícil,"Nervios, pero creo que lo puedo hacer sin prob...",Ninguna en específico
6,Diagramas de flujo y relacionar con cosas,Bien,Abandono y trabaje solo,Más o menos,No tengo problema,Un poco nervioso,Relajarme y caminar
7,Explicando con adjetivos y ejemplos pasados a...,Siento más presión,"Hacer el trabajo yo solo, haciendo el trabajo ...",De alguna manera si,Depende la conexion,Depende que emoción tenga en el momento,DTPA
8,Intermedio,Muy cómoda,"Organización, creando un calendario",Si,Muy fácil,Mal,Técnicas de respiración
9,"a veces es complejo, pero la mayoría de veces ...","con un equipo que conozco, aunque en su mayorí...",casi siempre es determinar un cronograma para ...,"nopi, me he tenido que forjar para llevar las ...","complicado, las personas me suelen caer bien, ...","cómodo, exponer suele ser complicado solo al i...",me como las uñas y en muy pocas ocasiones term...


In [6]:
# Aplicar la función de orden y luego la conversión a string
for df_id, df in dataframes.items():
    dataframes[df_id] = reordenar_columnas(df)  # Reordenar columnas
    dataframes[df_id] = convertir_a_string(dataframes[df_id])  # Convertir a string excepto la primera columna


In [7]:
# Iterar sobre los DataFrames en el diccionario `dataframes`
for i, df in dataframes.items():
    # Acceder al nombre de la segunda columna
    segunda_columna = df.columns[2]  # La segunda columna tiene índice 1
    print(f"Encabezado de la i columna en el DataFrame {i}: {segunda_columna}")


Encabezado de la i columna en el DataFrame 1: ¿Cuál es tu mayor preocupación al terminar tus estudios?
Encabezado de la i columna en el DataFrame 2: 1.  ¿Cuánto dinero generas al mes?
Encabezado de la i columna en el DataFrame 3: Dirección de correo electrónico
Encabezado de la i columna en el DataFrame 4: Dirección de correo electrónico
Encabezado de la i columna en el DataFrame 5: 2. Edad:
Encabezado de la i columna en el DataFrame 6: ¿Cuál es tu promedio actual?
Encabezado de la i columna en el DataFrame 7: ¿Cuál es tu edad?


# Limpieza de datos

## Funciones

In [8]:
# Función para clasificar una respuesta
def clasificar_respuesta(respuesta, respuestas_esperadas):
    # Asegurarse de que la respuesta sea una cadena
    if isinstance(respuesta, str):
        respuesta = respuesta.lower()
    else:
        respuesta = str(respuesta).lower()  # Convertir cualquier tipo a cadena y luego en minúsculas

    # Ahora comparar con las palabras clave
    for categoria, palabras_clave in respuestas_esperadas.items():
        for palabra in palabras_clave:
            if palabra.lower() in respuesta:  # Compara sin distinguir mayúsculas/minúsculas
                return categoria
    return "No Clasificado"  # Si no se encuentra ninguna coincidencia

def procesar_dataframe(df, respuestas_esperadas_df, preguntas_especiales):
    # Guardar el orden original de las columnas
    orden_columnas = df.columns.tolist()

    # Separar preguntas normales y especiales
    preguntas_normales = []
    preguntas_especiales_ = {'correos': [], 'cantidades': []}

    for pregunta in df.columns[2:]:  # Ignorar las primeras dos columnas
        if pregunta in preguntas_especiales['correos']:
            preguntas_especiales_['correos'].append(pregunta)
        elif pregunta in preguntas_especiales['cantidades']:
            preguntas_especiales_['cantidades'].append(pregunta)
        else:
            preguntas_normales.append(pregunta)

    # Procesar preguntas normales
    df_normal = df[preguntas_normales].copy()
    for pregunta in preguntas_normales:
        pregunta_respuestas_esperadas = respuestas_esperadas_df.get(pregunta, {})
        df_normal[pregunta] = df_normal[pregunta].apply(lambda x: clasificar_respuesta(x, pregunta_respuestas_esperadas))

    # Procesar preguntas especiales
    df_especial = df[preguntas_especiales_['correos'] + preguntas_especiales_['cantidades']].copy()

    for columna in preguntas_especiales_['correos']:
        df_especial[columna] = df_especial[columna].apply(lambda x: x if '@' in str(x) else None)


    for columna in preguntas_especiales_['cantidades']:
        df_especial[columna] = df_especial[columna].apply(
            lambda x: float(re.sub(r"[^0-9.]", "", x)) if isinstance(x, str) and re.sub(r"[^0-9.]", "", x).replace('.', '', 1).isdigit() else None
        )

    # Unir todo
    df_clean = pd.concat([df.iloc[:, :2], df_normal, df_especial], axis=1)

    # Reordenar las columnas para que coincidan con el original
    df_clean = df_clean[orden_columnas]

    return df_clean





## Respuestas esperadas

In [9]:
def generar_diccionario_respuestas_esperadas(dataframes):
    # Diccionario donde se almacenarán las preguntas por dataframe
    respuestas_esperadas = {}

    for df_id, df in dataframes.items():
        # Crear un diccionario para cada dataframe con el id correspondiente
        respuestas_esperadas[df_id] = {}

        # Recorrer las columnas de las preguntas (omite las primeras dos columnas: fecha y número de cuenta)
        for pregunta in df.columns[2:]:
            # Agregar la pregunta al diccionario y dejar la lista de respuestas vacía para completar
            respuestas_esperadas[df_id][pregunta] = {}

    return respuestas_esperadas

responses=generar_diccionario_respuestas_esperadas(dataframes)
# Diccionario principal con las respuestas esperadas por dataframe

In [10]:
responses

{1: {'¿Cuál es tu mayor preocupación al terminar tus estudios?': {},
  '¿Qué factores influyen más en tu decisión sobre qué camino seguir después de egresar?': {},
  '¿Cuál es tu situación actual respecto a la elección de un camino profesional después de egresar?': {},
  '¿Qué tipo de apoyo crees que te ayudaría más en esta etapa de decisión?': {},
  '¿Qué habilidades o conocimientos crees que son esenciales para tomar una decisión sobre tu especialización o carrera profesional después de egresar?': {},
  '¿En qué medida consideras que las prácticas profesionales o proyectos extracurriculares influirán en tu decisión de que camino seguir?': {},
  '¿Cómo visualizas tu carrera profesional dentro de 5 años? ¿Qué tipo de trabajo o proyectos te gustaría estar realizando?': {},
  '¿Qué es lo que más valoras en un posible empleo o área de especialización después de terminar tu carrera?': {},
  '¿Qué tan útil consideras el material de estudio de la carrera para conseguir trabajo?': {},
  '¿Sie

In [11]:
# Definir las preguntas especiales
preguntas_especiales = {
    'correos': [
        # Las que contienen correos
        'Dirección de correo electrónico'

    ],
    'cantidades': [  # Las que contienen cantidades
        '1.  ¿Cuánto dinero generas al mes?',
        '2. ¿Cuánto gastas mensualmente considerando todos tus gastos?',
        '3. ¿Cuánto dinero logras ahorrar al mes?',
        '¿Cuánto tiempo tardas en llegar a casa después de la clase? (Escribe tu respuesta en minutos)',
        '2. Edad:',
        '6. ¿Cuánto tiempo en minutos dura tu recorrido a la Facultad?',
        '¿Cuál es tu edad?',
    ]
}



In [21]:
respuestas_esperadas={
    1: {
      '¿Cuál es tu mayor preocupación al terminar tus estudios?': {
        "Trabajo": ["Trabajo", "Empleo"],
        "Obtener el título": ["Título", "Graduación","titulo","titularme"],
        "Otro": ["Otro"],
        "Ninguna":["Nada", "Ninguna", "ninguna"]
        },

      '¿Qué factores influyen más en tu decisión sobre qué camino seguir después de egresar?': {
          "Económico": ["Económico", "Finanzas"],
          "Social": ["Social", "Entorno"],
          "Otro": ["Otro"]
      },
      '¿Cuál es tu situación actual respecto a la elección de un camino profesional después de egresar?': {
          "Confusión": ["Confusión", "Indeciso"],
          "Seguro": ["Seguro", "Decidido"],
          "Otro": ["Otro"]
      },
      '¿Qué tipo de apoyo crees que te ayudaría más en esta etapa de decisión?': {
          "Económico": ["Económico", "Apoyo financiero"],
          "Orientación profesional": ["Asesoría", "Mentoría"],
          "Otro": ["Otro"]
      },
      '¿Qué habilidades o conocimientos crees que son esenciales para tomar una decisión sobre tu especialización o carrera profesional después de egresar?': {
          "Conocimiento del campo laboral": ["Campo laboral", "Áreas de especialización", "Futuro laboral", "Ofertas de vacantes"],
          "Autoconocimiento y toma de decisiones": ["Autoconocimiento", "Saber qué quiero", "Expresar lo que sé hacer"],
          "Habilidades técnicas específicas": ["Matemáticas aplicadas", "Programación", "Estadística", "Creación de CV"],
          "Habilidades blandas y estratégicas": ["Pensamiento crítico", "Comunicación", "Networking", "Idiomas"],
          "Adaptabilidad y aprendizaje continuo": ["Aprendizaje", "Disposición a aprender"]
      },
      '¿En qué medida consideras que las prácticas profesionales o proyectos extracurriculares influirán en tu decisión de que camino seguir?': {
          "Mucho": ["Mucho", "Bastante"],
          "Poco": ["Poco", "Nada"]
      },
      '¿Cómo visualizas tu carrera profesional dentro de 5 años? ¿Qué tipo de trabajo o proyectos te gustaría estar realizando?': {
          "En un puesto de liderazgo": ["Ser jefe", "Gerente"],
          "Con un empleo estable": ["Teniendo trabajo", "Empleo fijo"],
          "No estoy seguro": ["No sé", "Indefinido"]
      },
      '¿Qué es lo que más valoras en un posible empleo o área de especialización después de terminar tu carrera?': {
          "Flexibilidad": ["Flexibilidad", "Horarios cómodos"],
          "Ambiente laboral": ["Ambiente laboral", "Cultura de empresa"],
          "Oportunidad de crecimiento": ["Crecimiento", "Desarrollo profesional"],
          "Otro": ["Otro"]
      },
      '¿Qué tan útil consideras el material de estudio de la carrera para conseguir trabajo?': {
          "Muy útil": ["Muy útil", "Relevante"],
          "Poco útil": ["Sin utilidad", "No aplicable"]
      },
      '¿Sientes que tu carrera universitaria te permitió desarrollar una red de contactos profesionales que te ha ayudado en tu camino laboral?': {
          "Si": ["Si"],
          "No": ["No"]
      },
      '¿Qué aspectos de tu formación académica crees que podrían mejorarse para preparar mejor a los futuros egresados?': {
          "Más habilidades prácticas": ["Mejores habilidades", "Más práctica"],
          "No estoy seguro": ["No sé","no se"]
      },
      '¿Consideras que el contenido de tu carrera estuvo actualizado con las necesidades actuales del mercado laboral?': {
          "Sí": ["Sí"],
          "No": ["No"],
          "Depende del área": ["Otro"]
      }
    },
  2: {
      '1.  ¿Cuánto dinero generas al mes?': {}, #No rellenar
      '2. ¿Cuánto gastas mensualmente considerando todos tus gastos?': {}, #No rellenar
      '3. ¿Cuánto dinero logras ahorrar al mes?': {}, #No rellenar
      '4. ¿Tienes dinero reservado para emergencias? Si sí, ¿cuánto aproximadamente?': {
          "Si": ["Si"],
          "No": ["No"]
      },
      '5. ¿Tu dinero tiene liquidez o está comprometido en inversiones o deudas?': {
          "Liquidez": ["Liquid"],
          "Deudas": ["Deudas"]
      },
      '6. ¿Cuentas con inversiones a corto plazo?': {
          "Si": ["Si"],
          "No": ["No"]
      },
      '7. ¿Cuentas con inversiones a largo plazo?': {
          "Si": ["Si"],
          "No": ["No"]
      },
      '8. ¿Tienes un plan personal de retiro o planeas tener uno en el futuro?': {
          "Si": ["Si"],
          "No": ["No"],
          "Futuro": ["futuro"]
      },
      '9. ¿Cuántas tarjetas de crédito tienes?': {
          "0":["Ninguna", "cero", "no tengo", "nada","zero", "Ninguna ", " Ninguna "],
          "1":["una", "uno", "1"],
          "2":["dos", "2"],
          "3":["tres", "3"],
          "4":["cuatro","4"]
      },
      '10. ¿Qué porcentaje de tu línea de crédito tienes disponible actualmente?': {
          "0%": ["0"],
          "25-50%": ["25, 50%"],
          "50-75%": ["50, 75%"],
          "75-100%": ["75", "100%"]
      }
      },
  3: {
      'Dirección de correo electrónico': {}, #No rellenar
      '2. ¿Cómo describirías tu capacidad para explicar ideas complejas a personas que no son expertas en tu área?': {
        "Buena": ["Buena", "Clara", "Explico con facilidad"],
        "Intermedia": ["Intermedio", "Suficientemente claro", "Moderada"],
        "Mala": ["Mala", "Dificultad para explicar"]
      },
      '3. ¿Qué tan cómodo te sientes trabajando en equipo?': {
        "Cómodo": ["Cómodo", "Me siento bien", "Fluidez"],
        "Incómodo": ["Incómodo", "Dificultades para colaborar"],
        "Neutral": ["Neutral", "Ni cómodo ni incómodo"]
      },
      '4. ¿Cuál ha sido tu mayor desafío al trabajar en equipo y cómo lo superaste? ': {
        "Comunicación": ["Comunicación", "Falta de claridad"],
        "Organización": ["Organización", "Planificación deficiente"],
        "Otro": ["Otro", "Conflictos interpersonales", "Descoordinación"]
      },
      '5.¿Sientes que la carrera de MAC te ha preparado adecuadamente para el trabajo en equipo?': {
        "Si": ["Si", "Bien preparado"],
        "No": ["No", "Falta"]
      },
      '6.  ¿Qué tan fácil o difícil te resulta establecer nuevas conexiones? ': {
          "Fácil": ["Fácil", "Abierto a nuevas relaciones", "Sociable"],
          "Intermedio": ["Intermedio", "Dependiendo de la situación"],
          "Difícil": ["Difícil", "Me cuesta conocer gente nueva"]
      },
      '7. ¿Cómo te sientes al tener que hablar frente a una gran audiencia?': {
          "Nervioso": ["Nervioso", "Ansioso", "Tenso"],
          "Cómodo": ["Cómodo", "Confianza"],
          "Otro": ["Otro", "Inseguro"]
      },
      '8. ¿Qué técnicas usas para manejar el estrés y mantener la calma en situaciones difíciles?': {
          "Ejercicios de respiración": ["Ejercicios de respiración", "Meditación", "Relajación"],
          "Técnicas de manejo de tiempo": ["Manejo del tiempo", "Organización", "Prioridades"],
          "Otro": ["Otro", "Ejercicio físico", "Desconectar", "Hablar con alguien"]
      },
      '9.  ¿Cómo te preparas para presentaciones orales o exposiciones frente a un público amplio? ': {
          "Liderazgo": ["Liderazgo", "Gestión de equipos", "Tomar decisiones"],
        "Comunicación": ["Comunicación", "Escucha activa", "Expresión clara"],
        "Otro": ["Otro", "Resolución de conflictos", "Trabajo bajo presión"]

      },
      '10. ¿Crees que las habilidades técnicas (programación, matemáticas, algoritmos) son suficientes para desempeñarte en el ámbito laboral sin necesidad de habilidades blandas?': {
          "Si": ["Si", "Las habilidades técnicas son lo más importante"],
          "No": ["No", "Las habilidades blandas también son fundamentales"]
      },
      '11. ¿Has aplicado alguna de las habilidades blandas aprendidas en estos cursos en tu vida académica o personal?': {
          "Si": ["Si", "Claro"],
          "No": ["No", "Negativo"]
      },
      '12. ¿Cuáles consideras que son las habilidades blandas más importantes para un profesional de Matemáticas Aplicadas y Computación? ': {
          "Trabajo en equipo": ["Trabajo en equipo", "Colaboración"],
          "Comunicación": ["Comunicación", "Expresión efectiva"],
          "Liderazgo": ["Liderazgo", "Gestión de equipos"],
          "Otro": ["Otro", "Pensamiento crítico", "Adaptabilidad"]
      },
      '13.  ¿Cuáles de las siguientes habilidades blandas crees que te faltan desarrollar más ': {
          "Liderazgo": ["Liderazgo", "Gestión de equipos", "Tomar decisiones"],
          "Comunicación": ["Comunicación", "Escucha activa", "Expresión clara"],
          "Otro": ["Otro", "Resolución de conflictos", "Trabajo bajo presión"]
      },
      '14.  ¿Qué tan preparado/a te sientes en términos de habilidades blandas para enfrentar un empleo después de graduarte? ': {
          "Preparado": ["Preparado", "Confío en mis habilidades"],
          "Intermedio": ["Intermedio", "Me falta algo de preparación"],
          "No preparado": ["No preparado", "Necesito más desarrollo"]
      },
      '15.  ¿Consideras que los estudiantes de MAC suelen subestimar la importancia de las habilidades blandas? ': {
          "Sí": ["Sí", "Se enfocan mucho en lo técnico"],
          "No": ["No", "Le dan la misma importancia a las dos"]
      }
      },
  4: {
      'Dirección de correo electrónico': {}, #No rellenar
      '¿Qué tanto afecta tener una materia de 18:00 a 20:00 hrs en viernes a tu desempeño académico?': {},
      ' ¿Qué tanto afecta tener una materia de 18:00 a 20:00 hrs en viernes a tu desempeño personal?': {},
      '¿Qué dificultades enfrentas para asistir y concentrarte en esta materia en ese horario? (Separa cada una por comas)': {},
      '¿Qué tanto impacta este horario en tu organización del tiempo para otras actividades académicas o laborales?': {},
      '¿Crees que el rendimiento en una materia como Minería de Datos se ve afectado por el horario? ¿Por qué? ': {},
      '¿Qué tanto afecta este horario a tu alimentación y descanso?': {},
      '¿Qué sugerencias darías para mejorar la experiencia de aprendizaje en esta materia considerando el horario? (Menciona las sugerencias separando por comas)': {},
      '¿Has considerado dejar esta materia debido al horario? ¿Por qué?': {},
      '¿Qué herramientas o métodos crees que podrían hacer más llevadera la clase en este horario? (Menciona las herramientas separando por comas)': {},
      '¿Cuál es tu principal medio de transporte para llegar y salir de la universidad?': {},
      '¿Has experimentado dificultades con el transporte debido al horario de la materia? Si es así, ¿cuáles?': {},
      '¿Cuánto tiempo tardas en llegar a casa después de la clase? (Escribe tu respuesta en minutos)': {}, #No rellenar
      '¿Te sientes seguro al transportarse después de las 20:00 hrs? Explica tu respuesta.': {},
      '¿Has tenido que modificar tus rutas o modos de transporte por la hora en que termina la clase?': {}
      },
  5: {
      '2. Edad:': {}, #No rellenar
      '3. Genero:': {},
      '4. Estado donde resides:': {},
      '5. ¿Actualmente estas laborando o realizando tu Servicio Social?': {},
      '6. ¿Cuánto tiempo en minutos dura tu recorrido a la Facultad?': {},#No rellenar
      '7.  ¿Que materias de la carrera te interesaron más las de área computacional o matemática? ': {},
      '8. ¿Tienes Automóvil?': {},
      '9. ¿En qué Área de especialidad de la carrera estás interesado?': {},
      '10. ¿Qué materia se te dificultó más en la carrera?': {},
      '11. ¿Cuales son las 3 materias que consideras que más te aportado más en tu elección de área de especialidad?': {},
      '12. ¿En cuantas y cuales materias elegiste a un profesor por "barquear" la materia?': {},
      '13. ¿Te consideras bueno trabajando en equipo? ¿por qué?': {},
      '14. ¿Te consideras perfeccionista?': {},
      '15. ¿Tus padres o alguna persona cercana se encuentra laborando en algo relacionado con el área de especialidad que elegiste? ': {},
      '16. ¿Te gustaría crecer en una empresa siempre con el mismo proyecto durante años o preferirias trabajar en varios proyectos de diferentes temas?': {},
      '17. ¿Qué persona consideras es tu modelo a seguir?': {},
      '18. ¿Consideras que tu equipo de computo es adecuado para el software requerido en la carrera?': {},
      '19. ¿Qué software utilizado en la carrera consideras el mas complejo de usar?': {},
      '20. ¿Qué software utilizado en la carrera consideras el mas fácil de usar?': {}
      },
  6: {
      '¿Cuál es tu promedio actual?': {}, #En conflicto
      '  ¿Cuántas horas dedicas al estudio semanalmente y cómo las distribuyes?  ': {},
      '¿Cuántas materias debes actualmente?': {},
      '¿Cuántas materias has reprobado durante la carrera?': {},
      '¿Cuántos años llevas en la carrera?': {},
      '¿Cuál es la raíz de que debas materias?': {},
      '¿Los profesores te han apoyado durante la carrera?': {},
      '  ¿Qué factores externos (familiares, laborales, sociales) afectan tu desempeño académico?  ': {},
      '  ¿Qué cambios o mejoras sugerirías en el sistema educativo para mejorar el rendimiento académico?  ': {},
      '  ¿Te parece que los exámenes reflejan de manera justa tu nivel de conocimiento? ¿Por qué?  ': {},
      '  ¿Cómo gestionas tu tiempo entre tus responsabilidades académicas y otras actividades?  ': {}
      },
  7: {
      '¿Cuál es tu edad?': {}, #No rellenar
      '¿En qué semestre te encuentras?': {
          "6to u 8vo semestre": ["6", "8"]
      },
      '¿Qué dispositivos electrónicos utilizas con mayor frecuencia?\n(Ej. teléfono móvil, tablet, computadora, etc.)': {
          "Teléfono móvil y computadora": ["teléfono móvil", "computadora"],
          "Teléfono móvil y tablet": ["teléfono móvil", "tablet"],
          "Teléfono móvil y computadora": ["teléfono móvil", "computadora"],
          "Tablet y computadora": ["tablet", "computadora"],
      },
      '¿Cuántas horas al día usas dispositivos electrónicos para fines personales?': {
          "3-6 horas": ["3-6"],
          "6-12 horas": ["6-12"],
          "Más de 12 horas": ["más"]
      }, #En conflicto
      '¿Utilizas dispositivos electrónicos durante las horas escolares? ¿Para qué actividades?': {
          "Si": ["Si"],
          "No": ["No"]
      },
      '¿Qué redes sociales usas regularmente?\n(Ej: Instagram, Facebook, TikTok, Twitter, etc.)': {
          "Instagram": ["Instagram"],
          "Facebook": ["Facebook"],
          "TikTok": ["TikTok"],
          "Twitter": ["Twitter"],
          "Ninguna": ["Ninguna"]
      },
      '¿Cuál es el principal motivo por el que utilizas las redes sociales?': {
          "Comunicación": ["Comunicación"],
        "Entretenimiento": ["Entretenimiento"],
        "Información": ["Información"]
      },
      '¿En qué momentos del día sueles acceder a las redes sociales?': {
            "Mañana": ["Mañana"],
            "Tarde": ["Tarde"],
            "Noche": ["Noche"],
            "Variado": ["Variado"]
      },
      '¿Utilizas las redes sociales mientras realizas tareas escolares o estudios? ¿Cómo influye en tu concentración?': {
          "Me distrae mucho": ["Me distrae mucho"],
          "No afecta": ["No afecta"],
          "Afecta moderadamente": ["Afecta moderadamente"]
      },
      '¿Consideras que el uso de redes sociales afecta tu rendimiento académico? Explica tu respuesta.': {
          "Si": ["Si"],
          "No afecta": ["No afecta","No"],
          "Depende": ["Depende"]
      },
      '¿Has notado cambios en tu capacidad de concentración o productividad al usar redes sociales?': {
          "Si": ["Si"],
          "No": ["No"]
      },
      '¿Crees que el uso excesivo de tecnología afecta la interacción con tus compañeros en el entorno escolar? Explica tu percepción.': {
          "Sí": ["Sí"],
          "No": ["No"]
      },
      '¿Estás al tanto de las políticas o normas sobre el uso de tecnología en tu escuela? ¿Las consideras suficientes?': {
          "Si": ["Si"],
          "No": ["No"]
      },
      '¿Recibes información o formación sobre cómo proteger tu privacidad y seguridad en línea? ¿Qué aspectos te gustaría reforzar?': {
          "Si": ["Si"],
          "No": ["No"]
      },
      '¿Has sido testigo o víctima de ciberacoso o situaciones inseguras en redes sociales?': {
          "Si": ["Si"],
          "No": ["No"]
      },
      '¿Te sientes preparado/a para hacer un uso responsable y seguro de la tecnología? ¿Por qué?': {
          "Si": ["Si"],
          "No": ["No"],
          "Parcialmente": ["Parcialmente"]
      },
      '¿Qué tipo de talleres o formación adicional te gustaría recibir en relación con el uso de tecnología y redes sociales?': {
          "Seguridad en linea": ["Si, sobre seguridad"],
          "Uso saludable de redes sociales": ["Si, sobre uso saludable"],
          "No interesado": ["No"]
      },
      '¿Cómo crees que las redes sociales influyen en tu estado de ánimo o bienestar emocional?': {
          "No afecta": ["No"],
          "Depende del contenido": ["Depende del contenido"],
          "Influye negativamente": ["Influye negativamente"]
      },
      '¿Consideras que el uso de redes sociales ha mejorado o dificultado la comunicación y relación con tus compañeros? Detalla tu experiencia': {
          "Mejorado": ["Mejorado"],
          "Dificultado": ["Dificultado"],
          "Sin cambio": ["Sin cambio"]
      },
      '¿Crees que el uso excesivo de tecnología puede contribuir al aislamiento o la desconexión social? Explica tu perspectiva': {
          "Si": ["Si"],
          "No": ["No"],
          "Depende": ["Depende"]
      },
      '¿Qué medidas o estrategias propondrías para promover un uso saludable de la tecnología en la escuela?': {
          "Regulación del tiempo": ["Regulación del tiempo"],
          "Conciencia sobre distracción": ["Conciencia sobre distracción"],
          "No es necesario": ["No es necesario"]
      },
      '¿Qué actividades o programas te gustaría que se implementaran para mejorar la educación digital y el manejo de redes sociales?': {
          "Talleres": ["Talleres"],
          "Charlas": ["Charlas"],
          "Campañas": ["Campañas"],
          "No interesado": ["No interesado"]
      }
      }
    }

## Clasificación de respuestas

In [22]:

# Iterar sobre los dataframes para procesarlos
dataframes_clean = {}
for df_id, df in dataframes.items():
    respuestas_esperadas_df = respuestas_esperadas.get(df_id, {})
    df_clean = procesar_dataframe(df, respuestas_esperadas_df, preguntas_especiales)
    dataframes_clean[df_id] = df_clean

In [23]:
dataframes_clean[1].iloc[:,:]

,Marca temporal,Número de cuenta,¿Cuál es tu mayor preocupación al terminar tus estudios?,¿Qué factores influyen más en tu decisión sobre qué camino seguir después de egresar?,¿Cuál es tu situación actual respecto a la elección de un camino profesional después de egresar?,¿Qué tipo de apoyo crees que te ayudaría más en esta etapa de decisión?,¿Qué habilidades o conocimientos crees que son esenciales para tomar una decisión sobre tu especialización o carrera profesional después de egresar?,¿En qué medida consideras que las prácticas profesionales o proyectos extracurriculares influirán en tu decisión de que camino seguir?,¿Cómo visualizas tu carrera profesional dentro de 5 años? ¿Qué tipo de trabajo o proyectos te gustaría estar realizando?,¿Qué es lo que más valoras en un posible empleo o área de especialización después de terminar tu carrera?,¿Qué tan útil consideras el material de estudio de la carrera para conseguir trabajo?,¿Sientes que tu carrera universitaria te permitió desarrollar una red de contactos profesionales que te ha ayudado en tu camino laboral?,¿Qué aspectos de tu formación académica crees que podrían mejorarse para preparar mejor a los futuros egresados?,¿Consideras que el contenido de tu carrera estuvo actualizado con las necesidades actuales del mercado laboral?
0,2025-02-25 20:30:04.522,422075345,No Clasificado,No Clasificado,No Clasificado,No Clasificado,Conocimiento del campo laboral,Mucho,No Clasificado,No Clasificado,No Clasificado,No,No Clasificado,No Clasificado
1,2025-02-26 03:56:02.340,318104263,Trabajo,No Clasificado,Confusión,No Clasificado,No Clasificado,Mucho,No Clasificado,No Clasificado,No Clasificado,Si,No Clasificado,No
2,2025-02-28 00:26:52.421,422068044,No Clasificado,No Clasificado,No Clasificado,Económico,Autoconocimiento y toma de decisiones,No Clasificado,No Clasificado,No Clasificado,No Clasificado,Si,No Clasificado,No
3,2025-02-28 17:14:19.785,317237748,Trabajo,No Clasificado,No Clasificado,No Clasificado,Habilidades técnicas específicas,Poco,No Clasificado,No Clasificado,Muy útil,No Clasificado,No Clasificado,Sí
4,2025-02-28 17:35:49.834,421087105,Trabajo,No Clasificado,No Clasificado,No Clasificado,No Clasificado,Mucho,No Clasificado,No Clasificado,No Clasificado,No,No Clasificado,No
5,2025-02-28 17:36:20.116,319289619,No Clasificado,No Clasificado,No Clasificado,No Clasificado,Habilidades blandas y estratégicas,No Clasificado,No Clasificado,Ambiente laboral,No Clasificado,Si,No Clasificado,No
6,2025-02-28 17:47:44.134,422087359,No Clasificado,Económico,No Clasificado,No Clasificado,No Clasificado,Mucho,No Clasificado,No Clasificado,No Clasificado,Si,No Clasificado,No Clasificado
7,2025-02-28 17:48:43.332,422079312,Obtener el título,No Clasificado,No Clasificado,No Clasificado,No Clasificado,No Clasificado,No Clasificado,No Clasificado,No Clasificado,No,No Clasificado,No Clasificado
8,2025-02-28 17:59:17.205,315124208,Trabajo,No Clasificado,No Clasificado,No Clasificado,No Clasificado,Mucho,No Clasificado,No Clasificado,No Clasificado,Si,No Clasificado,No
9,2025-02-28 18:08:24.654,319024791,No Clasificado,Económico,No Clasificado,No Clasificado,No Clasificado,No Clasificado,No Clasificado,Ambiente laboral,No Clasificado,No Clasificado,Más habilidades prácticas,Sí


In [20]:
dataframes[1]

,Marca temporal,Número de cuenta,¿Cuál es tu mayor preocupación al terminar tus estudios?,¿Qué factores influyen más en tu decisión sobre qué camino seguir después de egresar?,¿Cuál es tu situación actual respecto a la elección de un camino profesional después de egresar?,¿Qué tipo de apoyo crees que te ayudaría más en esta etapa de decisión?,¿Qué habilidades o conocimientos crees que son esenciales para tomar una decisión sobre tu especialización o carrera profesional después de egresar?,¿En qué medida consideras que las prácticas profesionales o proyectos extracurriculares influirán en tu decisión de que camino seguir?,¿Cómo visualizas tu carrera profesional dentro de 5 años? ¿Qué tipo de trabajo o proyectos te gustaría estar realizando?,¿Qué es lo que más valoras en un posible empleo o área de especialización después de terminar tu carrera?,¿Qué tan útil consideras el material de estudio de la carrera para conseguir trabajo?,¿Sientes que tu carrera universitaria te permitió desarrollar una red de contactos profesionales que te ha ayudado en tu camino laboral?,¿Qué aspectos de tu formación académica crees que podrían mejorarse para preparar mejor a los futuros egresados?,¿Consideras que el contenido de tu carrera estuvo actualizado con las necesidades actuales del mercado laboral?
0,2025-02-25 20:30:04.522,422075345,Comer,El dinero y la felicidad principalmente,Ya tengo un camino pensado y empezado,Tener transporte propio,Tener conocimiento sobre el campo laboral y có...,bastante,Ser director en la empresa en la que trabajo p...,Las actividades diarias y el salario,"No tanto, ya que todo se puede conseguir en el...",No tanto hasta el momento,Acercar más opciones de prácticas profesionale...,a veces
1,2025-02-26 03:56:02.340,318104263,No encontrar trabajo,Si voy a tener un buen sueldo,"Me siento indeciso, la mayoría de las áreas me...",Tener prácticas de diferentes áreas,Conocer muy bien de que trata cada area de esp...,Influyen mucho,"Me visualizo teniendo un trabajo estable, trab...",Que sea flexible con el área de trabajo y que ...,Medianamente útil,Si.,Tener materias de habilidades blandas.,No.
2,2025-02-28 00:26:52.421,422068044,quedarme sin casa ni posibilidad de alimento,oportunidades que aparezcan,hacer una maestría,económico,saber que pasos tomar y tener un mayor autocon...,"demasiado, da una pequeña muestra del mundo la...",-,tiempo y hacer algo que me apasione,mucho,"si, siempre se dedico a esa interacción",Una mayor relación laboral con las empresas y ...,"no, pero hay muchos cursos extras que lo compl..."
3,2025-02-28 17:14:19.785,317237748,No llegar a conseguir un buen empleo,Mi familia y que esté cerca de mi casa,Buscando buenas opciones de trabajo,Alguna persona de fuera que me oriente,Las mates aplicadas junto con la programación,"Poco, ya que he hecho varios proyectos","Ser el líder de una empresa o jefe de área, re...",Que valga más como persona que el trabajo que ...,Muy útil,Sí,Llegar más preparados y conociendo un poco más...,"Sí, por supuesto."
4,2025-02-28 17:35:49.834,421087105,no tener trabajo,el salario mensual,necesito tener mi titulo,conferencias,saber que quieres,mucho,homeoffice,el salario,nada util,no,ser más practico,no
5,2025-02-28 17:36:20.116,319289619,El nivel de inglés que cuento actualmente,En la confianza de mis conocimientos y sí dich...,Encontrar un puesto que se ajuste a mi área de...,Mayor cantidad de ferias del empleo en la facu...,"Pensamiento critico, buenas comunicación, domi...",Tendrían un gran impacto,Me gustaría tener una plaza en Oracle como des...,Ambiente laboral y oportunidad de crecimiento ...,Relativamente útil,Si,La mejora y actualización de los temarios y la...,En su mayoría no
6,2025-02-28 17:47:44.134,422087359,El rumbo de la sociedad,"Sociales, económicos, emocionales, interperson...",posponiendo la decisión,Ninguno,Si pudiera conocer la mejor elección sería muy...,Bastante,Me gustaría desarrollar software,La posibilidad de escalar,Bastante,Si,No lo se,Relativamen